# Exercise 2

This notebook serves as a comprehensive solution to Exercise 2 of the VU Machine Learning course (Summer Semester 2025). The primary objective of this exercise is to deepen our understanding of Neural Networks (NNs) by implementing them using various approaches and conducting a thorough comparative analysis. All approaches are applied to the [Polish Bankruptcies Dataset](https://archive.ics.uci.edu/dataset/365/polish+companies+bankruptcy+data) as well as the [Second Dataset]().

Throughout this notebook, we will:

- Implement a Neural Network framework from scratch: The architecture, backward and forward propagation and the entire network are built within the **nn** folder in this repo.

- Implement the same Neural Network using PyTorch: We leverage PyTorch's standard functions to create an equivalent NN, showcasing a more conventional approach to NN development.

- Utilize an LLM tool for NN implementation: Using ChatGPT 4o to generate another version of the NN from scratch, allowing for a direct comparison of code structure, design choices, and potential differences with our custom implementation.

- Investigate and experiment with NN configurations: We explore various hyperparameters, including different activation functions, numbers of layers, and nodes per layer, using a grid search approach to find optimal values.

- Analyze performance and resource usage: We calculate the total number of learnable parameters and the virtual RAM consumed by our instantiated NNs.

- Conduct a detailed comparison: The core of this notebook involves comparing the performance, efficiency, and implementation details across our custom-built NN, the PyTorch version, and the LLM-generated code. We discuss findings related to classification performance metrics and the insights gained from each implementation method.

### Setup and Imports

In [6]:
%pip install -q ucimlrepo

You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from scipy.io import arff
import requests
import io
from ucimlrepo import fetch_ucirepo

from  nn.nn import NN
from nn.layer import Layer
from nn.functions import *

## MNIST Dataset

In [2]:
mnist = pd.read_csv("train.csv")
split = int(len(mnist)*0.8)

y_train, y_test = mnist["label"].values[:split].astype(int), mnist["label"].values[split:].astype(int)
X_train, X_test = mnist.drop("label", axis=1).values[:split], mnist.drop("label", axis=1).values[split:]

y_train_encoded = one_hot_encoding(y_train, 10)
y_test_encoded = one_hot_encoding(y_test, 10)

In [3]:
layers = [
    Layer(input_size=X_train.shape[1], output_size=10, activation_function='softmax'),
    # Layer(input_size=10, output_size=10, activation_function='softmax'),
]

# Initialize the neural network
nn = NN(layers=layers, num_classes = 10, activation_function='softmax', loss_function='cross_entropy')

# Train the network
epochs = 5
nn.train(X_train, y_train_encoded, epochs=epochs, batch_size=100, learning_rate=0.1, verbose=True, visualize=True)

 20%|██        | 1/5 [00:02<00:09,  2.38s/it]

Epoch 1/5 ============ Loss: 0.050


 40%|████      | 2/5 [00:04<00:06,  2.32s/it]

Epoch 2/5 ============ Loss: 0.038


 60%|██████    | 3/5 [00:06<00:03,  1.99s/it]

Epoch 3/5 ============ Loss: 0.037


 80%|████████  | 4/5 [00:08<00:01,  1.97s/it]

Epoch 4/5 ============ Loss: 0.035


100%|██████████| 5/5 [00:10<00:00,  2.01s/it]

Epoch 5/5 ============ Loss: 0.034


In [4]:
nn.evaluate(X_test, y_test_encoded)

{'loss': np.float64(2.5090297915988944),
 'accuracy': np.float64(0.9091666666666667),
 'precision': np.float64(0.9085492565199148),
 'recall': np.float64(0.908163941302714),
 'f1_score': np.float64(0.9078221584260732)}

## Polish Bankruptcy Dataset

### Preprocessing Steps

In [5]:
data = fetch_ucirepo(id=365)

bancrupcy_df = data.data.original

print(bancrupcy_df.shape)
print(bancrupcy_df.columns)
print(bancrupcy_df.head())

(43405, 66)
Index(['year', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20',
       'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30',
       'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40',
       'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50',
       'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60',
       'A61', 'A62', 'A63', 'A64', 'class'],
      dtype='object')
   year        A1       A2       A3      A4       A5       A6        A7  \
0     1  0.200550  0.37951  0.39641  2.0472  32.3510  0.38825  0.249760   
1     1  0.209120  0.49988  0.47225  1.9447  14.7860  0.00000  0.258340   
2     1  0.248660  0.69592  0.26713  1.5548  -1.1523  0.00000  0.309060   
3     1  0.081483  0.30734  0.45879  2.4928  51.9520  0.14988  0.092704   
4     1  0.187320  0.61323  0.22960  1.4063  -7.3128  0.18732  0.187320   

        A

In [6]:
column_names = {
    "A1": "net_profit_total_assets",
    "A2": "total_liabilities_total_assets",
    "A3": "working_capital_total_assets",
    "A4": "current_assets_short_term_liabilities",
    "A5": "cash_securities_receivables_short_term_liabilities_ratio",
    "A6": "retained_earnings_total_assets",
    "A7": "EBIT_total_assets",
    "A8": "book_value_equity_total_liabilities",
    "A9": "sales_total_assets",
    "A10": "equity_total_assets",
    "A11": "gross_profit_financial_expenses_total_assets",
    "A12": "gross_profit_short_term_liabilities",
    "A13": "gross_profit_depreciation_sales",
    "A14": "gross_profit_interest_total_assets",
    "A15": "total_liabilities_gross_profit_depreciation_ratio",
    "A16": "gross_profit_depreciation_total_liabilities",
    "A17": "total_assets_total_liabilities",
    "A18": "gross_profit_total_assets",
    "A19": "gross_profit_sales",
    "A20": "inventory_sales_ratio",
    "A21": "sales_n_sales_n_minus_1",
    "A22": "profit_operating_activities_total_assets",
    "A23": "net_profit_sales",
    "A24": "gross_profit_3_years_total_assets",
    "A25": "equity_share_capital_total_assets",
    "A26": "net_profit_depreciation_total_liabilities",
    "A27": "profit_operating_activities_financial_expenses",
    "A28": "working_capital_fixed_assets",
    "A29": "logarithm_total_assets",
    "A30": "total_liabilities_cash_sales_ratio",
    "A31": "gross_profit_interest_sales",
    "A32": "current_liabilities_cost_products_sold_ratio",
    "A33": "operating_expenses_short_term_liabilities",
    "A34": "operating_expenses_total_liabilities",
    "A35": "profit_sales_total_assets",
    "A36": "total_sales_total_assets",
    "A37": "current_assets_inventories_long_term_liabilities",
    "A38": "constant_capital_total_assets",
    "A39": "profit_sales_sales_ratio",
    "A40": "current_assets_inventory_receivables_short_term_liabilities",
    "A41": "total_liabilities_operating_profit_depreciation_ratio",
    "A42": "profit_operating_activities_sales",
    "A43": "rotation_receivables_inventory_turnover_days",
    "A44": "receivables_sales_ratio",
    "A45": "net_profit_inventory",
    "A46": "current_assets_inventory_short_term_liabilities",
    "A47": "inventory_cost_products_sold_ratio",
    "A48": "EBITDA_total_assets",
    "A49": "EBITDA_sales",
    "A50": "current_assets_total_liabilities",
    "A51": "short_term_liabilities_total_assets",
    "A52": "short_term_liabilities_cost_products_sold_ratio",
    "A53": "equity_fixed_assets",
    "A54": "constant_capital_fixed_assets",
    "A55": "working_capital",
    "A56": "sales_cost_products_sold_sales_ratio",
    "A57": "current_assets_inventory_short_term_liabilities_sales_gross_profit_depreciation_ratio",
    "A58": "total_costs_total_sales",
    "A59": "long_term_liabilities_equity",
    "A60": "sales_inventory",
    "A61": "sales_receivables",
    "A62": "short_term_liabilities_sales_ratio",
    "A63": "sales_short_term_liabilities",
    "A64": "sales_fixed_assets"
}

bancrupcy_df.rename(columns=column_names, inplace=True)
bancrupcy_df.head()


,year,net_profit_total_assets,total_liabilities_total_assets,working_capital_total_assets,current_assets_short_term_liabilities,cash_securities_receivables_short_term_liabilities_ratio,retained_earnings_total_assets,EBIT_total_assets,book_value_equity_total_liabilities,sales_total_assets,...,sales_cost_products_sold_sales_ratio,current_assets_inventory_short_term_liabilities_sales_gross_profit_depreciation_ratio,total_costs_total_sales,long_term_liabilities_equity,sales_inventory,sales_receivables,short_term_liabilities_sales_ratio,sales_short_term_liabilities,sales_fixed_assets,class
0,1,0.200550,0.37951,0.39641,2.0472,32.3510,0.38825,0.249760,1.33050,1.1389,...,0.121960,0.39718,0.87804,0.001924,8.4160,5.1372,82.658,4.4158,7.4277,0
1,1,0.209120,0.49988,0.47225,1.9447,14.7860,0.00000,0.258340,0.99601,1.6996,...,0.121300,0.42002,0.85300,0.000000,4.1486,3.2732,107.350,3.4000,60.9870,0
2,1,0.248660,0.69592,0.26713,1.5548,-1.1523,0.00000,0.309060,0.43695,1.3090,...,0.241140,0.81774,0.76599,0.694840,4.9909,3.9510,134.270,2.7185,5.2078,0
3,1,0.081483,0.30734,0.45879,2.4928,51.9520,0.14988,0.092704,1.86610,1.0571,...,0.054015,0.14207,0.94598,0.000000,4.5746,3.6147,86.435,4.2228,5.5497,0
4,1,0.187320,0.61323,0.22960,1.4063,-7.3128,0.18732,0.187320,0.63070,1.1559,...,0.134850,0.48431,0.86515,0.124440,6.3985,4.3158,127.210,2.8692,7.8980,0


In [7]:
# ------------------------------------------------------
# Dataset preparation — starting after the original loading
# ------------------------------------------------------

# Make a copy to avoid touching the original loaded dataset
bankruptcy_df_base = bancrupcy_df.copy()

# Separate features and target
X = bankruptcy_df_base.drop(columns=['class', 'year'])  # Drop 'class' and 'Year' for now
y = bankruptcy_df_base['class']

### NN Modeling and Evaluation

In [8]:
train_test_split_index = int(len(X) * 0.8)
X_train = X[:train_test_split_index].values
X_test = X[train_test_split_index:].values
y_train = y[:train_test_split_index].values
y_test = y[train_test_split_index:].values
y_train_encoded = one_hot_encoding(y_train, 2)
y_test_encoded = one_hot_encoding(y_test, 2)

In [15]:
layers = [
    Layer(input_size=X_train.shape[1], output_size=10, activation_function='sigmoid'),
    Layer(input_size=10, output_size=4, activation_function='sigmoid'),
    Layer(input_size=4, output_size=2, activation_function='sigmoid'),
]

# Initialize the neural network
nn = NN(layers=layers, num_classes = 2, activation_function='sigmoid', loss_function='mean_squared_error')

# Train the network
epochs = 20
nn.train(X_train, y_train_encoded, epochs=epochs, batch_size=1000, learning_rate=0.1, verbose=True, visualize=True)

 10%|█         | 2/20 [00:00<00:02,  6.89it/s]

Epoch 1/20 ============ Loss: nan
Epoch 2/20 ============ Loss: nan


 20%|██        | 4/20 [00:00<00:02,  7.46it/s]

Epoch 3/20 ============ Loss: nan
Epoch 4/20 ============ Loss: nan


 30%|███       | 6/20 [00:00<00:02,  6.12it/s]

...

Epoch 16/20 ============ Loss: nan
Epoch 17/20 ============ Loss: nan


 95%|█████████▌| 19/20 [00:03<00:00,  5.93it/s]

Epoch 18/20 ============ Loss: nan
Epoch 19/20 ============ Loss: nan


100%|██████████| 20/20 [00:03<00:00,  6.03it/s]

Epoch 20/20 ============ Loss: nan


In [16]:
nn.evaluate(X_test, y_test_encoded)

{'loss': np.float64(nan),
 'accuracy': np.float64(0.8934454555926736),
 'precision': np.float64(0.4467227277963368),
 'recall': np.float64(0.5),
 'f1_score': np.float64(0.47186226196994585)}

## 2nd Dataset

### Preprocessing Steps

### NN Modeling and Evaluation